Importing the required libraries.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
import tensorflow as tf

Reading the dataset as input, and splitting the "Cabin" feature into three separate features, ensuring that "Transported" feature is at the last, (Change the directory location as required).

In [30]:
dataset = pd.read_csv("C:/Users/soume/OneDrive/Desktop/Space Titanic/train.csv")
dataset[["Deck", "Cabin_num", "Side"]] = dataset["Cabin"].str.split("/", expand=True)
cols_at_end = ['Transported']
dataset = dataset[[c for c in dataset if c not in cols_at_end] 
        + [c for c in cols_at_end if c in dataset]]

Storing the dependent and independent features.

In [31]:
dataset = dataset.drop('Cabin', axis=1)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Using Simple Imputer to replace the missing values in categrorical features with the most frequent values, and dropping the "Name" column.

In [32]:
imputer1 = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imputer1.fit(X[:, [1, 2, 3, 4, 6, 12, 14]])
X[:, [1, 2, 3, 4, 6, 12, 14]] = imputer1.transform(X[:, [1, 2, 3, 4, 6, 12, 14]])
X = np.delete(X, 11, 1)

Converting the values in "Cabin_num" from string to float.

In [33]:
for x in range(0, len(X)):
    X[x][12] = float(X[x][12])

Using Simple Imputer to replace the missing values in numerical features with their mean values.

In [34]:
imputer2 = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer2.fit(X[:, [4,6,7,8,9,10,12]])
X[:,[4,6,7,8,9,10,12]]  = imputer2.transform(X[:, [4,6,7,8,9,10,12]])

Using Label Encoder to apply label encoding on "Cryo" and "VIP" features, and removing the original versions of these features.

In [35]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
Cryo = X[:, 2]
VIP = X[:, 5]
Cryo = le1.fit_transform(Cryo)
VIP  = le2.fit_transform(VIP)
Cryo = Cryo.reshape(len(Cryo), 1)
VIP = VIP.reshape(len(VIP), 1)
X = np.hstack((X, Cryo))
X = np.hstack((X, VIP))
X = np.delete(X, 2, 1)
X = np.delete(X, 4, 1)

Removing the "PassengerID" column.

In [36]:
X = np.delete(X, 0, 1)

Using OneHotEncoder to apply One Hot Encoding on "HomePlanet" and "Destination" features, and avoiding the dummy variable trap by removing one dummy variable for each.

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct1 = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
X = np.array(ct1.fit_transform(X))
X = np.delete(X, 0, 1)
ct2 = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [2])], remainder = 'passthrough')
X = np.array(ct2.fit_transform(X))
X = np.delete(X, 0, 1)

Applying Label Encoding on the target feature.

In [38]:
le = LabelEncoder()
y = le.fit_transform(y)

Applying Label Encoding on "Side" feature.

In [39]:
Side = X[:, 12]
Side = le3.fit_transform(Side)
Side = Side.reshape(len(Side), 1)
X = np.hstack((X, Side))
X = np.delete(X, 12, 1)

Applying One Hot Encoding on "Deck" feature.

In [40]:
ct3 = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [10])], remainder = 'passthrough')
X = np.array(ct3.fit_transform(X))
X = np.delete(X, 0, 1)

Applying standardisation on independent features using Standard Scalar

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

Creating and training an ANN on input data.

In [42]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation="softsign"))
ann.add(tf.keras.layers.Dense(units=6, activation="softsign"))
ann.add(tf.keras.layers.Dense(units=6, activation="softsign"))
ann.add(tf.keras.layers.Dense(units=6, activation="softsign"))
ann.add(tf.keras.layers.Dense(units=6, activation="softsign"))
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

ann.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
ann.fit(X, y, batch_size=32, epochs=110)

Epoch 1/110
272/272 [==============================] - 1s 723us/step - loss: 0.5834 - accuracy: 0.7187
Epoch 2/110
272/272 [==============================] - 0s 743us/step - loss: 0.4714 - accuracy: 0.7913
Epoch 3/110
272/272 [==============================] - 0s 755us/step - loss: 0.4537 - accuracy: 0.7943
Epoch 4/110
272/272 [==============================] - 0s 747us/step - loss: 0.4427 - accuracy: 0.7997
Epoch 5/110
272/272 [==============================] - 0s 719us/step - loss: 0.4363 - accuracy: 0.8018
Epoch 6/110
272/272 [==============================] - 0s 700us/step - loss: 0.4310 - accuracy: 0.8008
Epoch 7/110
272/272 [==============================] - 0s 718us/step - loss: 0.4272 - accuracy: 0.8033
Epoch 8/110
272/272 [==============================] - 0s 727us/step - loss: 0.4235 - accuracy: 0.8017
Epoch 9/110
272/272 [==============================] - 0s 786us/step - loss: 0.4214 - accuracy: 0.8024
Epoch 10/110
272/272 [==============================] - 0s 693us/step - l

Applying the same data preprocessing as above on the test dataset features.

In [43]:
dataset = pd.read_csv("C:/Users/soume/OneDrive/Desktop/Space Titanic/test.csv")
dataset[["Deck", "Cabin_num", "Side"]] = dataset["Cabin"].str.split("/", expand=True)
dataset = dataset.drop('Cabin', axis=1)
X = dataset.iloc[:, :].values
passengerId = X[:, 0]
X[:, [1, 2, 3, 4, 6, 12, 14]] = imputer1.transform(X[:, [1, 2, 3, 4, 6, 12, 14]])
X = np.delete(X, 11, 1)
for x in range(0, len(X)):
    X[x][12] = float(X[x][12])
X[:,[4,6,7,8,9,10,12]]  = imputer2.transform(X[:, [4,6,7,8,9,10,12]])
Cryo = X[:, 2]
VIP = X[:, 5]
Cryo = le1.transform(Cryo)
VIP  = le2.transform(VIP)
Cryo = Cryo.reshape(len(Cryo), 1)
VIP = VIP.reshape(len(VIP), 1)
X = np.hstack((X, Cryo))
X = np.hstack((X, VIP))
X = np.delete(X, 2, 1)
X = np.delete(X, 4, 1)
X = np.delete(X, 0, 1)
X = np.array(ct1.transform(X))
X = np.delete(X, 0, 1)
X = np.array(ct2.transform(X))
X = np.delete(X, 0, 1)
Deck = X[:, 12]
Deck = le3.transform(Deck)
Deck = Deck.reshape(len(Deck), 1)
X = np.hstack((X, Deck))
X = np.delete(X, 12, 1)
X = np.array(ct3.transform(X))
X = np.delete(X, 0, 1)
X = sc.transform(X)

Using the ANN to predict target feature values.

In [44]:
y_pred = ann.predict(X)
y_pred = (y_pred>0.5)
y = np.empty(len(y_pred), dtype=object)
for x in range(0, len(y_pred)):
    if y_pred[x]==1:
        y[x] = "True"
    else:
        y[x] = "False"

134/134 [==============================] - 0s 557us/step


Creating a new csv file containing predictions, (Change the directory location as required).

In [45]:
dataframe1=pd.DataFrame(passengerId)
dataframe2=pd.DataFrame(y)
dataframe3=pd.concat([dataframe1,dataframe2],axis=1)
dataframe3.columns=['PassengerId','Transported']
dataframe3.to_csv("C:/Users/soume/OneDrive/Desktop/Space Titanic/titanic_ans1.csv", index=False)